# Sentinel Types - Distinguishing Absence from None

Sentinel types provide type-safe alternatives to `None` for representing missing or unset values. Lionherd defines two sentinel types:

**Core Sentinels:**
- **`Undefined`**: Field or key entirely missing from namespace (never-set)
- **`Unset`**: Key present but value not provided (pending)

**Why Not Just Use `None`?**
- `None` is a valid value in many contexts (nullable fields)
- Cannot distinguish `field=None` from `field not provided`
- APIs need to differentiate "don't update" from "set to null"

**Key Features:**
- **Singleton Pattern**: Safe identity checks with `is`
- **Falsy**: Evaluate to `False` in boolean context
- **Type Guards**: Narrow `MaybeSentinel[T]` to `T`
- **Immutable**: Preserve identity across copy/deepcopy/pickle

In [1]:
from __future__ import annotations

from copy import copy, deepcopy

from lionherd_core.types import (
    MaybeSentinel,
    MaybeUndefined,
    MaybeUnset,
    Undefined,
    Unset,
    is_sentinel,
    not_sentinel,
)

## 1. Basic Usage - Undefined vs Unset

**`Undefined`**: Represents a field that doesn't exist (missing key in dict, never-set attribute)

**`Unset`**: Represents a field that exists but has no value yet (pending input, optional parameter)

In [2]:
# Undefined - field doesn't exist
config = {"host": "localhost", "port": 8080}
username = config.get("username", Undefined)
print(f"Username: {username}")
print(f"Is Undefined: {username is Undefined}")
print(f"Type: {type(username).__name__}")

Username: Undefined
Is Undefined: True
Type: UndefinedType


In [3]:
# Unset - field exists but no value provided
def create_user(name: str, email: str | Unset = Unset) -> dict:
    user = {"name": name}
    if email is not Unset:
        user["email"] = email
    return user


# Email not provided - key omitted
user1 = create_user("Alice")
print(f"User without email: {user1}")

# Email provided as None - key included with None value
user2 = create_user("Bob", email=None)
print(f"User with None email: {user2}")

# Email provided - key included with value
user3 = create_user("Charlie", email="charlie@example.com")
print(f"User with email: {user3}")

User without email: {'name': 'Alice'}
User with None email: {'name': 'Bob', 'email': None}
User with email: {'name': 'Charlie', 'email': 'charlie@example.com'}


## 2. Sentinel vs None Comparison

Demonstrate the three-state distinction: value provided, `None` provided, nothing provided.

In [4]:
def update_profile(
    user_id: int, bio: str | None | Unset = Unset, avatar: str | None | Unset = Unset
) -> dict:
    """Update user profile. Unset = don't update, None = clear field, value = set field."""
    updates = {}

    if bio is not Unset:
        updates["bio"] = bio  # Could be None (clear) or string (set)

    if avatar is not Unset:
        updates["avatar"] = avatar  # Could be None (clear) or string (set)

    return {"user_id": user_id, "updates": updates}


# Don't update anything
result1 = update_profile(1)
print(f"No updates: {result1}")

# Clear bio (set to None)
result2 = update_profile(1, bio=None)
print(f"Clear bio: {result2}")

# Set bio, clear avatar
result3 = update_profile(1, bio="Software Engineer", avatar=None)
print(f"Set bio, clear avatar: {result3}")

No updates: {'user_id': 1, 'updates': {}}
Clear bio: {'user_id': 1, 'updates': {'bio': None}}
Set bio, clear avatar: {'user_id': 1, 'updates': {'bio': 'Software Engineer', 'avatar': None}}


## 3. Singleton Behavior

Sentinels are singletons - only one instance exists. Use `is` for identity checks, not `==`.

In [5]:
# Import creates same instance
from lionherd_core.types import (
    Undefined as Undef1,
    Undefined as Undef2,
)

print(f"Undefined is Undef1: {Undefined is Undef1}")
print(f"Undef1 is Undef2: {Undef1 is Undef2}")
print(f"id(Undefined) == id(Undef1): {id(Undefined) == id(Undef1)}")

Undefined is Undef1: True
Undef1 is Undef2: True
id(Undefined) == id(Undef1): True


In [6]:
# Copy and deepcopy preserve identity
copied = copy(Undefined)
deepcopied = deepcopy(Unset)

print(f"copy(Undefined) is Undefined: {copied is Undefined}")
print(f"deepcopy(Unset) is Unset: {deepcopied is Unset}")

copy(Undefined) is Undefined: True
deepcopy(Unset) is Unset: True


In [7]:
# Falsy in boolean context
print(f"bool(Undefined): {bool(Undefined)}")
print(f"bool(Unset): {bool(Unset)}")

# Safe in conditional expressions
value = Undefined
if not value:
    print("Sentinel is falsy - condition triggered")

# But still distinguishable from None/False
print(f"Undefined is None: {Undefined is None}")
print(f"Unset is False: {Unset is False}")

bool(Undefined): False
bool(Unset): False
Sentinel is falsy - condition triggered
Undefined is None: False
Unset is False: False


## 3b. Union Type Syntax (Python 3.10+)

Sentinels support modern Python union syntax using the `|` operator.

In [8]:
from typing import get_args

# Union syntax works in both directions
type1 = str | Unset
type2 = Unset | str
type3 = int | Undefined
type4 = Undefined | int

print("Union Syntax Examples:")
print(f"str | Unset: {type1}")
print(f"Unset | str: {type2}")
print(f"int | Undefined: {type3}")
print(f"Undefined | int: {type4}")

# Complex unions with multiple types
type5 = str | None | Unset
print(f"\nComplex union (str | None | Unset): {type5}")
print(f"  Type args: {get_args(type5)}")


# Works in function signatures (already shown in examples above)
def api_update(value: str | None | Unset = Unset) -> str:
    """Demonstrates union syntax in function signature."""
    if value is Unset:
        return "not provided"
    elif value is None:
        return "explicitly null"
    else:
        return f"value: {value}"


print("\nFunction annotation test:")
print(f"  api_update() = {api_update()}")
print(f"  api_update(None) = {api_update(None)}")
print(f"  api_update('hello') = {api_update('hello')}")

# Singleton identity preserved
print("\nSingleton identity preserved:")
print(f"  Unset is Unset: {Unset is Unset}")
print(f"  Undefined is Undefined: {Undefined is Undefined}")

Union Syntax Examples:
str | Unset: typing.Union[str, lionherd_core.types._sentinel.UnsetType]
Unset | str: typing.Union[lionherd_core.types._sentinel.UnsetType, str]
int | Undefined: typing.Union[int, lionherd_core.types._sentinel.UndefinedType]
Undefined | int: typing.Union[lionherd_core.types._sentinel.UndefinedType, int]

Complex union (str | None | Unset): typing.Union[str, NoneType, lionherd_core.types._sentinel.UnsetType]
  Type args: (<class 'str'>, <class 'NoneType'>, <class 'lionherd_core.types._sentinel.UnsetType'>)

Function annotation test:
  api_update() = not provided
  api_update(None) = explicitly null
  api_update('hello') = value: hello

Singleton identity preserved:
  Unset is Unset: True
  Undefined is Undefined: True


## 4. Type Safety with Type Guards

The `not_sentinel()` function narrows types for static type checkers.

In [9]:
def process_value(value: MaybeSentinel[int]) -> int:
    """Type checker knows value is int after not_sentinel check."""
    if not_sentinel(value):
        # Type narrowed: value is int here
        return value * 2
    else:
        # value is Undefined | Unset here
        return 0


print(f"process_value(5): {process_value(5)}")
print(f"process_value(Undefined): {process_value(Undefined)}")
print(f"process_value(Unset): {process_value(Unset)}")

process_value(5): 10
process_value(Undefined): 0
process_value(Unset): 0


In [10]:
# Type aliases for clearer signatures
def get_config(key: str) -> MaybeUndefined[str]:
    """Returns str or Undefined if key missing."""
    config = {"host": "localhost"}
    return config.get(key, Undefined)


def optional_param(value: MaybeUnset[int] = Unset) -> str:
    """Parameter can be int or Unset (not provided)."""
    if value is Unset:
        return "not provided"
    return f"value: {value}"


print(f"get_config('host'): {get_config('host')}")
print(f"get_config('port'): {get_config('port')}")
print(f"optional_param(): {optional_param()}")
print(f"optional_param(42): {optional_param(42)}")

get_config('host'): localhost
get_config('port'): Undefined
optional_param(): not provided
optional_param(42): value: 42


## 5. Helper Functions

`is_sentinel()` and `not_sentinel()` provide flexible checking with options for `None` and empty collections.

In [11]:
# Basic sentinel checking
values = [42, None, Undefined, Unset, "", [], {}]

print("Default behavior (only Undefined/Unset):")
for val in values:
    result = is_sentinel(val)
    print(f"  is_sentinel({val!r:20}): {result}")

Default behavior (only Undefined/Unset):
  is_sentinel(42                  ): False
  is_sentinel(None                ): False
  is_sentinel(Undefined           ): True
  is_sentinel(Unset               ): True
  is_sentinel(''                  ): False
  is_sentinel([]                  ): False
  is_sentinel({}                  ): False


In [12]:
# Treat None as sentinel
print("\nWith none_as_sentinel=True:")
for val in [42, None, Undefined, Unset]:
    result = is_sentinel(val, none_as_sentinel=True)
    print(f"  is_sentinel({val!r:20}, none_as_sentinel=True): {result}")


With none_as_sentinel=True:
  is_sentinel(42                  , none_as_sentinel=True): False
  is_sentinel(None                , none_as_sentinel=True): True
  is_sentinel(Undefined           , none_as_sentinel=True): True
  is_sentinel(Unset               , none_as_sentinel=True): True


In [13]:
# Treat empty collections as sentinel
print("\nWith empty_as_sentinel=True:")
for val in [42, "", [], {}, set(), Undefined]:
    result = is_sentinel(val, empty_as_sentinel=True)
    print(f"  is_sentinel({val!r:20}, empty_as_sentinel=True): {result}")


With empty_as_sentinel=True:
  is_sentinel(42                  , empty_as_sentinel=True): False
  is_sentinel(''                  , empty_as_sentinel=True): True
  is_sentinel([]                  , empty_as_sentinel=True): True
  is_sentinel({}                  , empty_as_sentinel=True): True
  is_sentinel(set()               , empty_as_sentinel=True): True
  is_sentinel(Undefined           , empty_as_sentinel=True): True


In [14]:
# Combined: None AND empty as sentinels
print("\nWith both flags:")
for val in [42, None, "", [], Undefined]:
    result = is_sentinel(val, none_as_sentinel=True, empty_as_sentinel=True)
    print(f"  is_sentinel({val!r:20}, both=True): {result}")


With both flags:
  is_sentinel(42                  , both=True): False
  is_sentinel(None                , both=True): True
  is_sentinel(''                  , both=True): True
  is_sentinel([]                  , both=True): True
  is_sentinel(Undefined           , both=True): True


## 6. Common Patterns - Default Values

Sentinels enable clearer distinction between "use default" vs "explicitly set to None".

In [15]:
class Connection:
    """Database connection with optional timeout."""

    def __init__(self, host: str, timeout: float | None | Unset = Unset, pool_size: int = 10):
        self.host = host
        self.pool_size = pool_size

        # Three states:
        # - Unset: use default (30.0)
        # - None: no timeout (wait forever)
        # - float: explicit timeout
        if timeout is Unset:
            self.timeout = 30.0  # Default
        else:
            self.timeout = timeout  # Could be None or float

    def __repr__(self) -> str:
        return f"Connection(host={self.host!r}, timeout={self.timeout}, pool_size={self.pool_size})"


# Use default timeout
conn1 = Connection("localhost")
print(f"Default: {conn1}")

# Explicitly no timeout (wait forever)
conn2 = Connection("localhost", timeout=None)
print(f"No timeout: {conn2}")

# Custom timeout
conn3 = Connection("localhost", timeout=5.0)
print(f"Custom: {conn3}")

Default: Connection(host='localhost', timeout=30.0, pool_size=10)
No timeout: Connection(host='localhost', timeout=None, pool_size=10)
Custom: Connection(host='localhost', timeout=5.0, pool_size=10)


## 7. Common Patterns - Partial Updates

API endpoints often need to distinguish "don't update field" from "set field to null".

In [16]:
from dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str
    email: str | None = None
    bio: str | None = None
    avatar_url: str | None = None


def patch_user(
    user: User,
    name: str | Unset = Unset,
    email: str | None | Unset = Unset,
    bio: str | None | Unset = Unset,
    avatar_url: str | None | Unset = Unset,
) -> User:
    """PATCH endpoint: only update provided fields."""
    if name is not Unset:
        user.name = name
    if email is not Unset:
        user.email = email
    if bio is not Unset:
        user.bio = bio
    if avatar_url is not Unset:
        user.avatar_url = avatar_url
    return user


# Initial state
user = User(id=1, name="Alice", email="alice@example.com", bio="Engineer", avatar_url="/avatar.png")
print(f"Initial: {user}\n")

# Update only name (other fields unchanged)
patch_user(user, name="Alice Smith")
print(f"After name update: {user}\n")

# Clear bio (set to None), leave others unchanged
patch_user(user, bio=None)
print(f"After clearing bio: {user}\n")

# Update email and avatar, leave name/bio unchanged
patch_user(user, email="alice.smith@example.com", avatar_url="/new_avatar.png")
print(f"After email/avatar update: {user}")

Initial: User(id=1, name='Alice', email='alice@example.com', bio='Engineer', avatar_url='/avatar.png')

After name update: User(id=1, name='Alice Smith', email='alice@example.com', bio='Engineer', avatar_url='/avatar.png')

After clearing bio: User(id=1, name='Alice Smith', email='alice@example.com', bio=None, avatar_url='/avatar.png')

After email/avatar update: User(id=1, name='Alice Smith', email='alice.smith@example.com', bio=None, avatar_url='/new_avatar.png')


## 8. Serialization Preservation

Sentinels preserve identity across pickle and unpickle.

In [17]:
import pickle

# Pickle and unpickle
data = {"status": "pending", "result": Unset, "error": Undefined}

pickled = pickle.dumps(data)
unpickled = pickle.loads(pickled)

print(f"Original: {data}")
print(f"Unpickled: {unpickled}")
print("\nIdentity preserved:")
print(f"  result is Unset: {unpickled['result'] is Unset}")
print(f"  error is Undefined: {unpickled['error'] is Undefined}")

Original: {'status': 'pending', 'result': Unset, 'error': Undefined}
Unpickled: {'status': 'pending', 'result': Unset, 'error': Undefined}

Identity preserved:
  result is Unset: True
  error is Undefined: True


## Summary Checklist

**Sentinel Essentials:**
- ✅ `Undefined` for missing fields (never existed)
- ✅ `Unset` for pending values (not yet provided)
- ✅ Distinguish from `None` (which is a valid value)
- ✅ Singleton pattern - safe `is` identity checks
- ✅ Falsy in boolean context
- ✅ Type guards narrow `MaybeSentinel[T]` to `T`
- ✅ Preserve identity across copy/deepcopy/pickle
- ✅ Helper functions with `none_as_sentinel` and `empty_as_sentinel` options

**Use Cases:**
- ✅ Default values vs explicit None
- ✅ PATCH endpoints (partial updates)
- ✅ Optional parameters (not provided vs provided as None)
- ✅ Configuration (missing keys vs null values)

**Next Steps:**
- See `Element` for base class using metadata
- See `Node` for content-bearing entities
- See `Progression` for multi-step field updates where Unset matters